## Setup notebook environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import nest_asyncio
import warnings

_ = load_dotenv(find_dotenv())
nest_asyncio.apply()
warnings.filterwarnings('ignore')

## Import constructed modules

In [3]:
__curdir__ = os.getcwd()

import sys
for folder in ["src", "tools"]:
    sys.path.append(
        os.path.join(__curdir__,
                    f"../{folder}")
    )

In [4]:
from calculator_tools import get_calculator_tool
from data_analysis_tools import get_da_tools
from fundamental_analysis_tools import get_fa_tools
from rag_tools import get_rag_tools
from search_tools import get_tavily_tool
from sec_tools import get_sec_tool
from technical_analysis_tools import get_ta_tools
from gmail_tool import get_gmail_tool

## Setup tools
> Check for loading bottlenecks using the time module

In [5]:
import time

start_time = time.time()
calculator_tool = get_calculator_tool()
print(f"Time to instantiate calculator tool: {time.time() - start_time} seconds")

start_time = time.time()
da_tool = get_da_tools()
print(f"Time to instantiate da tool: {time.time() - start_time} seconds")

start_time = time.time()
fa_tool = get_fa_tools()
print(f"Time to instantiate fa tool: {time.time() - start_time} seconds")

start_time = time.time()
textbook_tool = get_rag_tools(
    prompt_path = "../tools/prompt.txt"
)
print(f"Time to instantiate rag tool: {time.time() - start_time} seconds")

start_time = time.time()
search_tool = get_tavily_tool()
print(f"Time to instantiate search tool: {time.time() - start_time} seconds")

start_time = time.time()
sec_tool = get_sec_tool()
print(f"Time to instantiate sec tool: {time.time() - start_time} seconds")

start_time = time.time()
ta_tool = get_ta_tools()
print(f"Time to instantiate ta tool: {time.time() - start_time} seconds")

start_time = time.time()
gmail_tool = get_gmail_tool()
print(f"Time to instantiate gmail tool: {time.time() - start_time} seconds")

Time to instantiate calculator tool: 0.0005362033843994141 seconds
Time to instantiate da tool: 0.004097700119018555 seconds
Time to instantiate fa tool: 0.0003039836883544922 seconds
Time to instantiate rag tool: 0.48343563079833984 seconds
Time to instantiate search tool: 0.0013489723205566406 seconds
Time to instantiate sec tool: 3.153048038482666 seconds
Time to instantiate ta tool: 0.0006189346313476562 seconds
Time to instantiate gmail tool: 0.00211334228515625 seconds


## Setup agents

In [6]:
import autogen
from autogen.agentchat.contrib.llamaindex_conversable_agent import (
    LLamaIndexConversableAgent
)
from llama_index.core import Settings
from llama_index.core.tools.tool_spec.base import BaseToolSpec
from llama_index.core.agent import (
    FunctionCallingAgentWorker,
    AgentRunner,
)
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding
from typing import List, Optional

### Setup crew

In [7]:
Settings.llm = BedrockConverse(
    model = "anthropic.claude-3-haiku-20240307-v1:0",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name = os.environ["AWS_DEFAULT_REGION"]
)
Settings.embed_model = BedrockEmbedding(
    model = "amazon.titan-embed-text-v1",
    aws_access_key_id = os.environ["AWS_ACCESS_KEY"],
    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_region_name = os.environ["AWS_DEFAULT_REGION"]
)

In [8]:
def get_agent(tools: List[BaseToolSpec],
              agent_name: str,
              agent_description: str,
              system_message: Optional[str] = None,
              human_input_mode: Optional[str] = "NEVER"):
    agent_worker = FunctionCallingAgentWorker.from_tools(
        tools = tools,
        llm = Settings.llm,
        verbose = True
    )
    agent = AgentRunner(
        agent_worker = agent_worker
    )
    return LLamaIndexConversableAgent(
        name = agent_name,
        llama_index_agent=agent,
        system_message = system_message,
        description = agent_description,
        human_input_mode = human_input_mode,
        max_consecutive_auto_reply = 2,
    )

In [22]:
data_analyst = get_agent(
    agent_name = "Principal_data_analyst",
    tools = [*da_tool, *calculator_tool],
    system_message = """You are an expert in statistics and helps customers
    develop data-driven insights from data analysis using statistical tools and
    methods to guide decision-making.""",
    agent_description = """This agent helps customers undertake statistical analysis
    of financial market data using methods such as correlations, compounded annual
    growth rate, etc.""",
)

In [23]:
technical_analyst = get_agent(
    agent_name = "Principal_technical_analyst",
    tools = [*ta_tool, *calculator_tool],
    system_message = """You are the top technical analyst of the field, adroit
    at crystallizing insights and investment strategies from stock data. Use tools
    to compute important technical analysis metrics to guide your investment 
    recommendations.""",
    agent_description="""This agent helps customers undertake technical analysis of
    financial market data using methods such as stochastic relative strength index,
    bollinger bands, ichimoku cloud, etc."""
)

In [24]:
fundamental_analyst = get_agent(
    agent_name="Principal_fundamental_analyst",
    tools = [
        fa_tool,
        *calculator_tool,
    ],
    system_message = """You are the top fundamental analyst of the field, adroit
    at crystallizing insights and investment strategies from stock data. Use tools
    to compute important fundamenta analysis metrics to guide your investment
    recommendations.""",
    agent_description="""This agent helps customers undertake fundamental analysis
    of the financial market data of companies by computing metrics such as P/E ratio."""
)

In [25]:
research_analyst = get_agent(
    agent_name="Principal_researcher",
    tools = [
        *search_tool,
        *sec_tool,
    ],
    system_message = """You are the top finance researcher of the field, adroit
    at crystallizing insights and investment strategies from
    close reading of SEC reports and research articles online.""",
    agent_description="""This agent helps customers undertake analysis of the 
    financial performance of companies by close reading of SEC reports and 
    research articles online."""
)

In [26]:
professor = get_agent(
    agent_name="Distinguished_professor_of_finance",
    tools = [
        *textbook_tool,
    ],
    system_message="""You are a distinguished professor of Finance with a 
    specialization in investment finance. Your role is to answer specific
    questions on finance and critically review the recommendations made by other agents.""",
    agent_description="""Answer technical definitions and double check recommendations made by
    other analysts. Check against the textbook if required to see if the recommendations made 
    are sound. Be critical. If there is room for improvement, suggest how the recommendations
    can be improved.
    """,
)

In [14]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="ALWAYS",
    code_execution_config=False
)

In [15]:
llm_config = {
    "model": os.environ["AZURE_OPENAI_GPT4O_DEPLOYMENT_NAME"],
    "api_key": os.environ["AZURE_OPENAI_API_KEY"],
    "base_url": os.environ["AZURE_OPENAI_ENDPOINT"],
    "api_type": "azure",
    "api_version": os.environ["AZURE_API_VERSION"]
}

In [27]:
# reporter = autogen.ConversableAgent(
#     name="Principal finance reporter",
#     llm_config = llm_config,
#     system_message="""You are a Pulitzer prize winning reporter adroit at 
#     distilling complex concepts to crystal clear insights easily understood by 
#     the layperson. You endeavor to help customers understand the investment 
#     recommendations put forth by your team.""",
#     description = """After all the information is available, write a compelling
#     report that puts forward your team's recommendation.  Ensure that the report's 
#     recommendations are supported by  evidence. Otherwise, route it to the manager 
#     or the professor with a recommendation of what further justification is needed, 
#     so that they can follow-up with the respective analyst."""
# )
reporter = get_agent(
    agent_name="Principal_finance_reporter",
    tools = [
        *gmail_tool,
    ],
    system_message="""You are a Pulitzer prize winning reporter adroit at 
    distilling complex concepts to crystal clear insights easily understood by 
    the layperson. You endeavor to help customers understand the investment 
    recommendations put forth by your team.""",
    agent_description = """Use this agent to write a report, draft emails and
    send emails."""
)

# Group chat 1: 1 group chat configuration
This does not work well with chainlit because the chainlit user proxy agent has a hard time digesting this

In [23]:
agents = [
    technical_analyst,
    data_analyst,
    fundamental_analyst,
    professor,
    reporter,
    user_proxy,
    research_analyst,
]

In [ ]:
groupchat = autogen.GroupChat(
    agents = agents,
    messages = [],
    max_round = 5000,
    allowed_or_disallowed_speaker_transitions = {
        user_proxy: [
            technical_analyst, 
            fundamental_analyst,
            data_analyst,
            research_analyst,
            professor,
            reporter,
        ],
        technical_analyst:[professor],
        fundamental_analyst: [professor],
        data_analyst: [professor],
        research_analyst: [user_proxy],
        professor: [user_proxy],
        reporter:[user_proxy],
    },
    enable_clear_history = True,
    speaker_transitions_type="allowed",
)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config = llm_config
)

In [26]:
task = """Undertake a fundamental analysis of Illumina? Should I invest in Illumina stocks?"""
chat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Undertake a fundamental analysis of Illumina? Should I invest in Illumina stocks?

--------------------------------------------------------------------------------

Next speaker: Principal fundamental analyst

Added user message to memory: Undertake a fundamental analysis of Illumina? Should I invest in Illumina stocks?
=== LLM Response ===
Okay, let's perform a fundamental analysis of Illumina to evaluate whether it would be a good investment:
=== Calling Function ===
Calling function: evaluate_fundamentals with args: {"ticker": "ILMN"}
=== Function Output ===
            Gross Margin (%)  Net Margin (%)  Current Ratio    ROA (%)  \
2023-12-31         60.923623      -25.777087       1.661783 -11.482544   
2022-12-31         64.834206      -96.073298       1.284169 -35.945152   
2021-12-31         69.686257       16.836058       2.482159   5.007557   
2020-12-31         68.014819       20.253165       3.603698   8.648649   

              ROE (%)  Quick Ratio 

In [29]:
task = "Why do companies have shares?"

chat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Why do companies have shares?

--------------------------------------------------------------------------------

Next speaker: Principal fundamental analyst

Added user message to memory: Why do companies have shares?
=== LLM Response ===
Companies have shares for a few key reasons:

1. Raise Capital: Issuing shares allows companies to raise capital by selling equity in the business to investors. This provides the company with funds to finance operations, invest in growth, or pursue other strategic initiatives.

2. Ownership Structure: Shares represent ownership in the company. The shareholders own a portion of the company proportional to the number of shares they hold. This ownership structure allows for the distribution of profits and decision-making power.

3. Liquidity: Publicly traded shares provide liquidity, allowing shareholders to buy and sell their ownership stakes on stock exchanges. This makes it easier for investors to enter and exit the company's

## Configuration 2:

In [45]:
groupchat = autogen.GroupChat(
    agents = [
        technical_analyst,
        fundamental_analyst,
        # professor,
    ],
    messages = [],
    max_round = 8,
    speaker_selection_method="round_robin",
    allow_repeat_speaker = False,
    # allowed_or_disallowed_speaker_transitions = {
    #     technical_analyst:[professor],
    #     fundamental_analyst: [professor],
    #     # professor: [technical_analyst, fundamental_analyst],
    # },
    enable_clear_history = True,
    speaker_transitions_type="allowed",
)
manager = autogen.GroupChatManager(
    groupchat = groupchat,
    llm_config = llm_config
)

In [46]:
writer = autogen.AssistantAgent(
    name = "writer",
    llm_config=llm_config,
    system_message = """
    You are a professional writer, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    """,
)
reviewer = autogen.AssistantAgent(
    name="Reviewer",
    llm_config=llm_config,
    system_message="""
    You are a compliance reviewer, known for your thoroughness and commitment to standards.
    Your task is to scrutinize content for any harmful elements or regulatory violations, ensuring
    all materials align with required guidelines.
    You must review carefully, identify potential issues, and maintain the integrity of the organization.
    Your role demands fairness, a deep understanding of regulations, and a focus on protecting against
    harm while upholding a culture of responsibility.
    """,
)

user = autogen.UserProxyAgent(
    name="User",
    human_input_mode="ALWAYS",
    # is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

def writing_message(
    recipient,
    messages,
    sender,
    config
):
    return f"Polish the content to make an engaging and nicely formatted blog post. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

In [47]:
assistant_1 = autogen.AssistantAgent(
    name="Assistant_1",
    llm_config = llm_config
)

nested_chat_queue = [
    {"recipient": manager, "summary_method": "reflection_with_llm"},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
    {"recipient": reviewer, "message": "Review the content provided.", "summary_method": "last_msg", "max_turns": 1},
    {"recipient": writer, "message": writing_message, "summary_method": "last_msg", "max_turns": 1},
]

assistant_1.register_nested_chats(
    nested_chat_queue,
    trigger = user,
)

In [48]:
res = user.initiate_chat(
    recipient = assistant_1,
    message = "Should I buy NVIDIA shares or Microsoft shares or both?"
)

User (to Assistant_1):

Should I buy NVIDIA shares or Microsoft shares or both?

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Assistant_1 (to chat_manager):

Should I buy NVIDIA shares or Microsoft shares or both?

--------------------------------------------------------------------------------

Next speaker: Principal_technical_analyst

Added user message to memory: Should I buy NVIDIA shares or Microsoft shares or both?
=== LLM Response ===
Here is a technical analysis to help evaluate NVIDIA and Microsoft stocks:
=== Calling Function ===
Calling function: analyse with args: {"ticker": "NVDA", "period": "1y"}
=== Function Output ===
                     field recommendation  \
0       adi_recommendation            buy   
1     aroon_recommendation           wait  